In [1]:
from keras import backend as K
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping, ModelCheckpoint
from Image_Generator import TextImageGenerator
from Model import get_Model
from parameter import *
K.set_learning_phase(0)

Using TensorFlow backend.
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ydh09\AppData\Roa

C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ydh09\AppData\Roaming\

In [2]:
import tensorflow as tf
tf.__version__
tf.debugging.set_log_device_placement(True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13179108165853101709
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6620742943
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3041412051846917595
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [ ]:
model = get_Model(training=True)

try:
    model.load_weights('LSTM+BN4--26--0.011.hdf5')
    #model.load_weights('LSTM+BN5--25--0.402.hdf5')
    print("...Previous weight data...")
except:
    print("...New weight data...")
    pass

train_file_path = './DB/train/'
tiger_train = TextImageGenerator(train_file_path, img_w, img_h, batch_size, downsample_factor)
tiger_train.build_data()

valid_file_path = './DB/test/'
tiger_val = TextImageGenerator(valid_file_path, img_w, img_h, val_batch_size, downsample_factor)
tiger_val.build_data()



In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:

ada = Adadelta()

early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=4, mode='min', verbose=1)
checkpoint = ModelCheckpoint(filepath='LSTM+BN5--{epoch:02d}--{val_loss:.3f}.hdf5', monitor='loss', verbose=1, mode='min', period=1)
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=30,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

In [5]:
!python Prediction.py

...Previous weight data...
Predicted: None  /  True: 서울 03두1472
Predicted: None  /  True: 서울 09거5978
label: A76sk15980
Predicted: 서울 76sk15980  /  True: 서울 10나1598
Predicted: 서울 75도6420  /  True: 서울 23아8422
Predicted: None  /  True: 서울 24허0231
Predicted: None  /  True: 서울 25아0837
Predicted: None  /  True: 서울 29나5276
Predicted: 서울 45수5470  /  True: 서울 36허6547
Predicted: 서울 07수9480  /  True: 서울 38두9048
Predicted: None  /  True: 서울 54모8905
Predicted: 서울 77나9890  /  True: 서울 55수9289
label: M66wh60280
Predicted: 전남 66wh60280  /  True: 서울 64모4628
Predicted: None  /  True: 서울 65수8503
Predicted: 서울 27누2550  /  True: 서울 87거7335
Predicted: None  /  True: 경기 00누1187
Predicted: 광주 77조3550  /  True: 경기 02로3485
Predicted: None  /  True: 경기 08보5938
Predicted: None  /  True: 경기 17누1211
label: O07th37260
Predicted: 전북 07th37260  /  True: 경기 23보3726
label: A77sn63280
Predicted: 서울 77sn63280  /  True: 경기 36가5928
Predicted: None  /  True: 경기 37우8452
Predicted: None  /  True: 경기 39로2308
Predicted: None  / 

Using TensorFlow backend.
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ydh09\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ydh09\AppData\Roa

Predicted: 경기 77서4280  /  True: 강원 77서4908
Predicted: 서울 77버6410  /  True: 강원 78바8341
Predicted: None  /  True: 강원 81무3240
Predicted: 광주 77수5560  /  True: 강원 83라5346
Predicted: None  /  True: 강원 85자0432
Predicted: None  /  True: 강원 90바8691
Predicted: None  /  True: 강원 94노7868
Predicted: None  /  True: 강원 96자6155
Predicted: 충남 37소4850  /  True: 충남 07오9315
Predicted: None  /  True: 충남 09오3597
Predicted: None  /  True: 충남 19부7525
Predicted: 전남 26사4270  /  True: 충남 26사1427
label: E44tk48980
Predicted: 충남 44tk48980  /  True: 충남 44다4698
Predicted: 전남 65주2680  /  True: 충남 49주2968
Predicted: None  /  True: 충남 49주8080
label: M24wn75540
Predicted: 전남 24wn75540  /  True: 충남 54루7264
Predicted: 전남 65부5930  /  True: 충남 60부6893
Predicted: None  /  True: 충남 63다4490
Predicted: None  /  True: 충남 65사3293
Predicted: None  /  True: 충남 70루5102
Predicted: None  /  True: 대전 01모2885
label: N07an98920
Predicted: 광주 07an98920  /  True: 대전 05두9892
Predicted: None  /  True: 대전 12고1966
Predicted: None  /  True: 대전 